# Exporting and Deploying with Torschscript

In [27]:
import os
import time
import cv2
import torch
import torchvision
import argparse
import numpy as np
from PIL import Image
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import logging
import torch
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from PIL import Image

In [19]:
#TODO: remove this stupidness.
os.chdir("..")

In [29]:
from roboflow import Roboflow
rf = Roboflow(api_key="gOwZbHTcuBdgjBYxl7Nk")
project = rf.workspace("sih-daaio").project("urban-small")
dataset = project.version(2).download("coco-segmentation")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to urban-small-2 in coco-segmentation:: 100%|█| 602/602 [00:00<00:00, 3055.13it/


In [49]:
dataset_name = "urban-small"
dataset_folder = "urban-small-2"

from detectron2.data.datasets import register_coco_instances
register_coco_instances(f"{dataset_name}_train", {}, f"./{dataset_folder}/train/_annotations.coco.json", f"./{dataset_folder}/train/")
register_coco_instances(f"{dataset_name}_val", {}, f"./{dataset_folder}/valid/_annotations.coco.json", f"./{dataset_folder}/valid/")
register_coco_instances(f"{dataset_name}_test", {}, f"./{dataset_folder}/test/_annotations.coco.json", f"./{dataset_folder}/test/")

AssertionError: Dataset 'urban-small_train' is already registered!

In [21]:
yaml_file = "weights/buildings_poc_cfg.yml"
model_weights = "weights/model_final.pth"

In [47]:
cfg = get_cfg()
cfg.merge_from_file(yaml_file)
cfg.MODEL.WEIGHTS = model_weights
cfg.DATASETS.TRAIN = (f"{dataset_name}_train",)
cfg.DATASETS.TEST = (f"{dataset_name}_test",)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.DEVICE = "cpu" 
predictor = DefaultPredictor(cfg)

[10/04 20:45:59 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from weights/model_final.pth ...


Skip loading parameter 'proposal_generator.rpn_head.conv.weight' to the model due to incompatible shapes: (256, 256, 3, 3) in the checkpoint but (1024, 1024, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.conv.bias' to the model due to incompatible shapes: (256,) in the checkpoint but (1024,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.weight' to the model due to incompatible shapes: (3, 256, 1, 1) in the checkpoint but (15, 1024, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.bias' to the model due to incompatible shapes: (3,) in the checkpoint but (15,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.weight' to the model due to incompatible s

In [44]:
p = cfg.dump()

In [42]:
data_loader = build_detection_test_loader(cfg, cfg.DATASETS.TEST[0])
first_batch = next(iter(data_loader))

WARNING [10/04 20:37:21 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/04 20:37:21 d2.data.datasets.coco]: Loaded 18 images in COCO format from ./urban-small-2/test/_annotations.coco.json
[10/04 20:37:21 d2.data.build]: Distribution of instances among all 8 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|   buildings   | 0            | Commercial-.. | 52           | Community Use | 0            |
| Industrial-.. | 48           |   Mixed Use   | 23           | Recreationa.. | 0            |
|  Residential  | 169          | Transport-I.. | 0            |               |              |
|     total     | 292          |               |              |               |              |
[10/04 20:37:21 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference:

In [ ]:
yaml_file = "weights/buildings_poc_cfg.yml"
model_weights = "weights/model_final.pth"
./build/torchscript_mask_rcnn output/model.ts input.jpg tracing

In [48]:
!detectron2/tools/deploy/export_model.py --config-file weights/buildings_poc_cfg.yml \
    --output ./output --export-method tracing --format torchscript \
    MODEL.WEIGHTS weights/model_final.pth \
    MODEL.DEVICE cpu

[10/04 20:46:07 detectron2]: Command line arguments: Namespace(format='torchscript', export_method='tracing', config_file='weights/buildings_poc_cfg.yml', sample_image=None, run_eval=False, output='./output', opts=['MODEL.WEIGHTS', 'weights/model_final.pth', 'MODEL.DEVICE', 'cpu'])
[W init.cpp:836] Warning: Use _jit_set_fusion_strategy, bailout depth is deprecated. Setting to (STATIC, 1) (function operator())
[10/04 20:46:08 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from weights/model_final.pth ...
Skip loading parameter 'proposal_generator.rpn_head.conv.weight' to the model due to incompatible shapes: (256, 256, 3, 3) in the checkpoint but (1024, 1024, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.conv.bias' to the model due to incompatible shapes: (256,) in the checkpoint but (1024,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal

In [ ]:
def inference(model, inputs):
    # use do_postprocess=False so it returns ROI mask
    inst = model.inference(inputs, do_postprocess=False)[0]
    return [{"instances": inst}]

isinstance(image, np.ndarray) == True
image_tensor = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
wrapper= TracingAdapter(predictor, inputs=[{"image": image_tensor}], inference_func=inference)